# Supplemental Figures: TIE Microscopy for Dynamic Microtubules
## Q. Tyrell Davis - 2017

### Figure S3: Offset Methods for Stabilizing  the 1D TIE in the Fourier Domain
* <strong>a</strong>) The quadratic (found in the Fourier domain TIE) demonstrates the instability of the TIE equation. As the spatial frequency goes to 0, the FD TIE will blow up. We can add a small global offset to the denominator to prevent division by 0, or as we see here we can add a small sinusoidal bump (a shifted Hanning function) to selectively offset the quadratic for low spatial frequencies. 
* <strong>b</strong>) The Fourier domain TIE demonstrates the effect of adding a Hanning bump (green) vs. a global offset (red). 
* <strong>c</strong>) Simulated signal + noise used in this figure. 
* <strong>d</strong>) Signal + noise recovered after convolution with <strong>b</strong>
* <strong>e</strong>) Recovered signal with simulated noise removed
To adjust width and amplitude of the Hanning bump, use the blue sliders at the bottom of the figure

### Figure S4: Exploring Offset Methods for Stabilizing TIE Solutions for a 2D Rod Phantom
Fourier domain Poisson solvers and recovered TIE micrographs for a simulated rod phantom.
* <strong>a</strong>) Fourier domain Poisson solver with a Hanning bump offset 
* <strong>b</strong>) Fourier domain Poisson solver with a global offset 
* <strong>c</strong>) Rod phantom phase map recovered with Hanning bump TIE solver
* <strong>d</strong>) Rod phantom phase map recovered with global offset TIE solver

Blue sliders can be used to adjust $w_0$ and offset amplitude. Although the signal-to-noise ratio is reported here according to the definition for MT micrographs described in [1], the optimized choice of parameters depends on one's objectives. Setting amplitude and $w_0$ aggressively can be used for high contrast images with MTs that are more easily recognizable to a human user or pattern-recognition software. If accurate optical path lengths are desired for quantitative analysis lesser offsets may be desirable. Furthermore, high amplitude and low $w_0$ Hanning bumps create an integrated Fourier band-pass filter in the TIE solver and should be minimized for quantitative images of larger phase objects such as cells


### Figure S5: Efffect of Parameter Choices on TIE Solutions for MT Micrographs
Fourier domain Poisson solvers and recovered TIE micrographs of microtubules.
* <strong>a</strong>) Fourier domain Poisson solver with a Hanning bump offset 
* <strong>b</strong>) Fourier domain Poisson solver with a global offset 
* <strong>c</strong>) Microtubules micrograph phase map recovered with Hanning bump TIE solver
* <strong>d</strong>) Microtubules micrograph phase map recovered with global offset TIE solver

The choice of parameters for the Hanning bump width and amplitude has effects the quality of the computed phase map. The left column displays the Fourier domain TIE solver filter and the resulting phase map, and the right column displays the Fourier domain solver using a global offset and the resulting phase map. This figure can be used to make decisions about when to use a global offset and when and what parameter settings to use with a Hanning bump.

To adjust width and amplitude of the Hanning bump, use the blue sliders at the bottom of the figure

### Figure S6: Comparison of VE-BF, TIEM, and Additional Computational Imaging Modes for Visualizing MTs
This is an interactive version of Figure 1: TIEM of microtubules and comparison to other imaging modes. Three examples of MT images are available for comparison, or the user can replace these with their own image stacks for comparison.

* <strong>a</strong>) Video-enhanced bright field micrograph of microtubules adhered to a coverslip (average of 42 frames with background subtracted), defocused below the sample plane 
* <strong>b</strong>)  VE-BF at the sample plane ($I_0$)
* <strong>c</strong>)  VE-BF  225 nm above the sample plane ($I_a$). <strong>d</strong>} TIE phase contrast micrograph of microtubules.
* <strong>d</strong>) TIE phase contrast micrograph of microtubules.
* <strong>e</strong>) The gradient of <strong>d</strong> mimicking DIC contrast with a y-axis shear.
* <strong>f</strong>) Gradient of <strong>d</strong> mimicking DIC contrast with a x-axis shear. Contrast in <strong>e</strong>) and <strong>f</strong> is essentially nonexistent for MTs parallel to the shear axis.
* <strong>g</strong>) The Laplacian ($\nabla^2[\phi(x,y)]$) of <strong>d</strong>. 
* <strong>h</strong>)  The intensity difference image ($I_a-I_b$). For micrographs of sufficiently thin phase objects, the first term in Eq. \ref{eq:productrule} goes to 0 and <strong>h</strong> and <strong>g</strong> are mathematically equivalent. 
* <strong>i</strong>) Through the eyepiece' (no background subtraction) view of the sample plane.

In [ ]:
# Import plotting essentials and necesary numpy/scipy
from matplotlib import pyplot as plt
import colormaps as cmaps
from scipy import misc
import numpy as np
# Sliders reference used: https://matplotlib.org/2.0.0/examples/widgets/slider_demo.html
from matplotlib.widgets import Slider, Button, RadioButtons
import scipy.ndimage
# Used for scale bars
import matplotlib.patches as patches


if (0):
    # If nbagg backend is set up it can be used for interactive inline figures
    %matplotlib nbagg

# Use viridis as the colormap
plt.register_cmap(name='viridis', cmap=cmaps.viridis)
# Adjust DPI settings to taste
myDPI = 80

In [ ]:
def TIEMMT(dI,px,hannScale,w0):
    # Estimate a solution to the transport of intensity equation and return
    # the resulting phase maps.
    # Inputs:
    # dI - First derivative of intensity with respect to defocus. Assuming this incorporates k0 and dz = (k0*dI)/dz
    # px - pixel size in microns (physical pixel size divided by magnification)
    # hannScale - scaling factor (amplitude) for Hanning bump
    # w0 - width of the Hanning bump
    # 
    # Returns:
    # Phase map and Fourier Domain TIE solvers with Hanning bump and global offset 
    # myTIEH, myTIEGO, phiHann, phiGo
    # Author: Q. Tyrell Davis 2017
    
    
    # Image dimensions
    Nx = len(dI[0,:])
    Ny = len(dI[:,0])
    
    X1 = np.arange(-Nx,Nx,1)
    Y1 = np.arange(-Ny,Ny,1)
    
    # set up the coordinates for the Fourier Domain
    myPadFactor = 2
    freqStepX = 2*np.pi / (Nx*myPadFactor)
    freqStepY = 2*np.pi / (Ny*myPadFactor)
   
    wX = np.arange(-np.pi+freqStepX,np.pi+freqStepX,freqStepX)
    wY = np.arange(-np.pi+freqStepY,np.pi+freqStepY,freqStepY)
    WX,WY = np.meshgrid(wX,wY) 
    wR = np.sqrt(WX**2+WY**2)
    
    ww = wR
    wR[wR >= np.pi/w0] = np.pi/(w0)
    # wR is used to define the Hanning Bump (set to make hBump value zero outside w0)
    # ww is used to calculate the TIE 
    
    # define the Hanning window, used to stabilize the FD TIE
    # w0 defines how wide, hannScale defines the amplitude
    hann= (hannScale)*(1+np.cos(wR*np.pi/w0))
    dI = dI
    # Transform dI/dz to the Fourier Domain, use zero padding to 2X dimensions
    DI = np.fft.fftshift(np.fft.fft2(dI,[Ny*myPadFactor, Nx*myPadFactor]))

    # Compute the phase
    # Define the Laplacian FD transform pair w/ Hanning bump or global offset
    myTIEGO = (1/ ((4*np.pi**2)*(ww**2+hannScale)))
    myTIEH = (1/ ((4*np.pi**2)*(ww**2+hann)))
    
    # compute phase maps (all caps are FD)
    PHI = - DI * myTIEH
    PHINH = - DI * myTIEGO
    phiHann = np.real(np.fft.ifft2(np.fft.ifftshift(PHI)))
    phiGo = np.real(np.fft.ifft2(np.fft.ifftshift(PHINH)))
    
    # crop the phase image ()
    phiHann = phiHann[0:Ny,0:Nx]
    phiGo = phiGo[0:Ny,0:Nx]
    
    # adjust phase map result so that all values are positive
    if (1):
        phiHann = phiHann - np.min(phiHann)
        phiGo = phiGo -np.min(phiGo)
        
    return phiHann, phiGo, myTIEH, myTIEGO 

In [ ]:
def TIEProp(myAmp,dz,k0,myPhi):
    # Compute a first derivative of intensity given an intensity, phase image, defocus distance, and wavenumber
    
    # Compute the TIE. 
    # For now ignoring pixel size
    myGradIx, myGradIy = np.gradient(myAmp)
    myGradI = myGradIx + myGradIy
    myGradPhix, myGradPhiy = np.gradient(myPhi)
    myGradPhi = myGradPhix+myGradPhiy
    myGradx, myGrady = np.gradient(myAmp*myGradPhi)
    myGrad = myGradx + myGrady
    
    #Add dI/dz
    newImg = myAmp+(dz/k0)*myGrad
    return newImg

In [ ]:
# This function is used for labeling subplots
def getAxisLimits(ax, scale=.8):#,indxX=0,indxY=1):
    return ax.get_xlim()[0]*scale, ax.get_ylim()[1]*scale
# This function is used for labeling subplots in S4/S5

def getAxisLimitsS4(ax, scale=.8):#,indxX=0,indxY=1):
    return ax.get_xlim()[0]*scale+myDPI/3, ax.get_ylim()[1]*scale+myDPI*1.75


def getAxisLimitsS5(ax, scale=.8):#,indxX=0,indxY=1):
    return ax.get_xlim()[0]*scale+myDPI/3, ax.get_ylim()[1]*scale+myDPI*3.5

In [ ]:
def updateFigS3(val):
    # Update HB parameters
    hanScale = samp1.val
    w0 = sfreq1.val
    
    
    # Calculate HB
    han = (hanScale/2)*(1-np.cos((np.pi*w/w0)-np.pi))
    for c in range(len(kx)):
        if np.abs(kx[c]) >= w0:
            han[c] = 0
    # Generate 1D Fourier domain TIE solver filters 
    phi2 = k0 / ((4*np.pi**2)*(kx**2+han))
    phi3 = k0 / ((4*np.pi**2)*(kx**2+hanScale))

    #Compute 1D "Phase" for Hanning bump and global offset
    myPhase1 = np.fft.ifft(mySN*(phi2))
    myNoise1 = np.fft.ifft(np.fft.fft(myNoise)*(phi2))
    myPhase2 = np.fft.ifft(mySN*(phi3))
    myNoise2 = np.fft.ifft(np.fft.fft(myNoise)*(phi3))
    
    # Clear figure axes (prevent legend entries and plots from piling up)
    myAxes[0,0].cla()
    myAxes[0,1].cla()
    myAxes[1,0].cla()
    myAxes[1,1].cla()
  
    
    # Plot the quadratic from the FD TIE denominator w/ and w/o Hanning bump offset
    myAxes[0,0].plot(kx,han,color=[0.95,0.35,0.15],lw = myLW,label='Hanning bump')
    #myAxes[0,0].plot(kx,kx**2*4*np.pi**2,'--',color=[0.2,0.2,0.2],lw = myLW,label='Quadratic') #label='Unmodified Denominator')
    #myAxes[0,0].plot(kx,han+kx**2*4*np.pi**2,c=[0.0,0.35,0.4],lw = myLW,label='Quadratic + Hanning')
    myAxes[0,0].plot(kx,kx**2,'--',color=[0.2,0.2,0.2],lw = myLW,label='Quadratic') #label='Unmodified Denominator')
    myAxes[0,0].plot(kx,han+kx**2,c=[0.0,0.35,0.4],lw = myLW,label='Quadratic + Hanning')
    
    myAxes[0,0].legend(loc=9,fontsize=fsz)
    myAxes[0,0].axis([-0.5, .5, -0.0015, .25])
       
    # plot the Fourier transform of a Laplacian ($F[\nabla**2]$)
    myAxes[0,1].plot(kx,phi1,'--',color=[0.2,0.2,0.2],lw = myLW,label=r'$\mathcal{F}[ \nabla^2 ]$')
    myAxes[0,1].plot(kx,phi2,color=[0.95,0.35,0.15],lw = myLW,label='w/ Hann')
    myAxes[0,1].plot(kx,phi3,c=[0,0,0],lw = myLW,label='w/ offset')
    myAxes[0,1].axis([-np.pi/2, np.pi/2, 0, 34])
    myAxes[0,1].legend(loc=1,fontsize=fsz)
   
    # plot the simulated signal line plot plus noise
    myAxes[1,0].plot(mySignalNoise/np.mean(mySignalNoise),'k',lw = myLW,label='Signal+noise ')
    myAxes[1,0].legend(loc=1,fontsize=fsz)
    myAxes[1,0].axis([0, myDim*2, 0, 7])
    
    # plot the inverse Fourier transform of the 1D signal+noise plot convolved with the FD Laplacian at a 
    # Subtract noise result from signal+noise result 
    mySig1 = np.abs(myPhase1)-np.abs(myNoise1)
    mySig2 = np.abs(myPhase2)-np.abs(myNoise2)
    myPk1 = np.max(mySig1)-np.min(mySig1)
    myPk2 = np.max(mySig2) - np.min(mySig2)
    std2 =  np.std(np.abs(myNoise2))
    std1 =  np.std(np.abs(myNoise1))
    # Plot
    myAxes[1,1].plot(np.abs(myPhase2),c=[0,0,0],lw = myLW,label='w/ offset')
    myAxes[1,1].plot(np.abs(myPhase1),color=[0.95,0.35,0.15],lw = myLW,label=' w/ Hanning')
    myAxes[1,1].legend(loc=1,fontsize=fsz)
    myAxes[1,1].axis([0, myDim*2, -1, 7])
   
   
    
    #clean up the axes,etc.
  
    myAxes[0,0].set_xlabel('a.u. index',fontsize=fsz)
    myAxes[0,0].set_ylabel('arbitrary units',fontsize=fsz)
    myAxes[0,1].set_xlabel('a.u. index',fontsize=fsz)
    myAxes[0,1].set_ylabel('arbitrary units',fontsize=fsz)
    myAxes[1,0].set_xlabel('a.u. index',fontsize=fsz)
    myAxes[1,0].set_ylabel('arbitrary units',fontsize=fsz)
    myAxes[1,1].set_xlabel('a.u. index',fontsize=fsz)
    myAxes[1,1].set_ylabel('arbitrary units',fontsize=fsz)
    
    #annotate subplots
    
    myAxes[0,0].annotate('a',xy=getAxisLimits(myAxes[0,0]),fontsize=fsz*3)
    myAxes[0,1].annotate('b',xy=getAxisLimits(myAxes[0,1],0.8),fontsize=fsz*3)
    myAxes[1,0].annotate('c',xy=getAxisLimits(myAxes[1,0],0.7),fontsize=fsz*3)
    myAxes[1,1].annotate('d',xy=getAxisLimits(myAxes[1,1],0.7),fontsize=fsz*3)
    
   
    myFig.suptitle("Figure S3: Offset Methods for Stabilizing  the 1D TIE in the Fourier Domain",fontsize=10+fsz)
    print("SNR G.O. = " +str(myPk2/std2))
    print("SNR Hann = " +str(myPk1/std1))
    
    
    plt.show()

In [ ]:
def updateTIEFigS4(val):
    # Update TIE phantom phase micrograph interactive figure.
    # Update the amplitude and width of the Hanning bump offset
    hannScale = samp2.val
    w0 = sfreq2.val
    
    # Get TIE phase map
    myPhiHann, myPhiGO, myPHI, myPHINH = TIEMMT(dI,px,hannScale,w0)
    
    # Display the Fourier transforms of the TIE
    myAxesS4[0,0].imshow(np.log(np.abs((myPHI))),cmap=cmaps.viridis)
    myAxesS4[0,0].set_title("Phantom TIE w/ Hanning Bump",fontsize=fsz*1.5)
    myAxesS4[0,1].imshow(np.log(np.abs((myPHINH))),cmap=cmaps.viridis)
    myAxesS4[0,1].set_title("Phantom TIE w/ Global Offset",fontsize=fsz*1.5)
    # Display TIE micrographs
    myAxesS4[1,0].imshow((np.real((myPhiHann))),cmap='gray')
    myAxesS4[1,1].imshow((np.real((myPhiGO))),cmap='gray')
  
    #Compute SNRs according to [1]: (pk-pk amplitude divided by standard deviation)
    myWind = int(myPhantDim /2)
    myBoxSize = 10
    mySNRHann = np.max(myPhiHann[myWind-myBoxSize:myWind+myBoxSize,myWind-myBoxSize:myWind+myBoxSize])-np.min(myPhiHann[myWind-myBoxSize:myWind+myBoxSize,myWind-myBoxSize:myWind+myBoxSize]) / np.std(myPhiHann[myWind-2*myBoxSize:myWind+2*myBoxSize,myWind+2*myBoxSize:myWind+4*myBoxSize])
    mySNRGO = np.max(myPhiGO[myWind-myBoxSize:myWind+myBoxSize,myWind-myBoxSize:myWind+myBoxSize])-np.min(myPhiGO[myWind-myBoxSize:myWind+myBoxSize,myWind-myBoxSize:myWind+myBoxSize]) / np.std(myPhiGO[myWind-2*myBoxSize:myWind+2*myBoxSize,myWind+2*myBoxSize:myWind+4*myBoxSize])
    # Report SNRs
    myAxesS4[1,0].set_title("SNR with Hanning bump = " + str(mySNRHann),fontsize=fsz*1.5)
    myAxesS4[1,1].set_title("SNR with global offset = "  + str(mySNRGO),fontsize=fsz*1.5)
 
    #add subplot labels
    myAxesS4[0,0].annotate('a',xy=getAxisLimitsS4(myAxesS4[0,0],0.7),fontsize=fsz*4,color='white')
    myAxesS4[0,1].annotate('b',xy=getAxisLimitsS4(myAxesS4[1,0],0.7),fontsize=fsz*4,color='white')
    myAxesS4[1,0].annotate('c',xy=getAxisLimitsS4(myAxesS4[1,0],0.7),fontsize=fsz*4,color='white')
    myAxesS4[1,1].annotate('d',xy=getAxisLimitsS4(myAxesS4[1,1],0.7),fontsize=fsz*4,color='white')
    # Remove numbers from axes
    for cx in range(0,2):
        for cy in range(0,2):
            myAxesS4[cx,cy].set_yticklabels([])
            myAxesS4[cx,cy].set_xticklabels([])
 
    myFigS4.suptitle("S4: Exploring Offset Methods for Stabilizing TIE Solutions for a 2D Rod Phantom",fontsize=10+fsz)
    # Make room for sliders
    plt.subplots_adjust(bottom=0.2)
    plt.show()

In [ ]:
def updateTIEFigS5(val):
    # Update TIE MT phase micrograph interactive figure.
    # Update the amplitude and width of the Hanning bump offset
    hannScale = samp2.val
    w0 = sfreq2.val
    
    # Get TIE phase map
    myPhiHann, myPhiGO, myPHI, myPHINH = TIEMMT(dI,px,hannScale,w0)

    # Display the Fourier transforms of the TIE
    myAxesS5[0,0].imshow(np.log(np.abs((myPHI))),cmap=cmaps.viridis)
    myAxesS5[0,0].set_title("TIEM w/ Hanning Bump",fontsize=fsz*1.5)
    myAxesS5[0,1].imshow(np.log(np.abs((myPHINH))),cmap=cmaps.viridis)
    myAxesS5[0,1].set_title("TIEM w/ Global Offset",fontsize=fsz*1.5)
    myAxesS5[1,0].imshow((np.real((myPhiHann))),cmap='gray')
    myAxesS5[1,1].imshow((np.real((myPhiGO))),cmap='gray')

    #add subplot labels
    myAxesS5[0,0].annotate('a',xy=getAxisLimitsS5(myAxesS5[0,0],0.7),fontsize=fsz*4,color='white')
    myAxesS5[0,1].annotate('b',xy=getAxisLimitsS5(myAxesS5[1,0],0.7),fontsize=fsz*4,color='white')
    myAxesS5[1,0].annotate('c',xy=getAxisLimitsS5(myAxesS5[1,0],0.7),fontsize=fsz*4,color='white')
    myAxesS5[1,1].annotate('d',xy=getAxisLimitsS5(myAxesS5[1,1],0.7),fontsize=fsz*4,color='white')
    
    #Remove the tick numbers
    for cx in range(0,2):
        for cy in range(0,2):
            myAxesS5[cx,cy].set_yticklabels([])
            myAxesS5[cx,cy].set_xticklabels([])

    
    myFigS5.suptitle("Fig S5: Efffect of Parameter Choices on TIE Solutions for MT Micrographs",fontsize=10+fsz)
    # Make room for sliders
    plt.subplots_adjust(bottom=0.2)
    plt.show()

In [ ]:
def updateTIEFigS6(hannScale,w0):

    myPhiHann, myPhiGO, myPHI, myPHINH = TIEMMT(dI,px,hannScale,w0)
    DICx, DICy = np.gradient(myPhiHann)
    
    del2 = scipy.ndimage.filters.laplace(myPhiHann)
    myIa = Ia - (myBG)
    myI0 = I0 - (myBG)
    myIb = Ib - (myBG)
    
    myAxesS6[0,0].imshow(myIb,cmap='gray',vmin=np.min(myIb)*myStretch, vmax=np.max(myIb)*myStretch)
    myAxesS6[0,1].imshow(myI0,cmap='gray',vmin=np.min(myI0)*myStretch, vmax=np.max(myI0)*myStretch)
    myAxesS6[0,2].imshow(myIa,cmap='gray',vmin=np.min(myIa)*myStretch, vmax=np.max(myIa)*myStretch)
    myAxesS6[1,0].imshow(myPhiHann,cmap='gray',vmin=np.max(myPhiHann)*(1-myStretch), vmax=np.max(myPhiHann)*myStretch)
    
    # Make it look like AFM! 
    #myAxesS6[1,0].imshow(myPhiHann,cmap='afmhot')
    myAxesS6[1,1].imshow(DICx,cmap='gray',vmin=np.min(DICx)*myStretch, vmax=np.max(DICx)*myStretch)
    myAxesS6[1,2].imshow(DICy,cmap='gray',vmin=np.min(DICy)*myStretch, vmax=np.max(DICy)*myStretch)
    myAxesS6[2,0].imshow(del2,cmap='gray',vmin=np.min(del2)*myStretch, vmax=np.max(del2)*myStretch)
    myAxesS6[2,1].imshow(dI,cmap='gray',vmin=np.min(dI)*myStretch, vmax=np.max(dI)*myStretch)
    myAxesS6[2,2].imshow(I0,cmap='gray',vmin=np.min(I0), vmax=np.max(I0))


    
    
    myScale = 0.5
    myAxesS6[0,0].annotate('a',xy=getAxisLimitsS4(myAxesS6[0,0],myScale),fontsize=fsz*3)
    myAxesS6[0,1].annotate('b',xy=getAxisLimitsS4(myAxesS6[0,1],myScale),fontsize=fsz*3)
    myAxesS6[0,2].annotate('c',xy=getAxisLimitsS4(myAxesS6[1,0],myScale),fontsize=fsz*3)
    myAxesS6[1,0].annotate('d',xy=getAxisLimitsS4(myAxesS6[1,1],myScale),color=[1,1,1],fontsize=fsz*3)
    myAxesS6[1,1].annotate('e',xy=getAxisLimitsS4(myAxesS6[2,0],myScale),fontsize=fsz*3)
    myAxesS6[1,2].annotate('f',xy=getAxisLimitsS4(myAxesS6[2,1],myScale),fontsize=fsz*3)
    myAxesS6[2,0].annotate('g',xy=getAxisLimitsS4(myAxesS6[1,1],myScale),fontsize=fsz*3)
    myAxesS6[2,1].annotate('h',xy=getAxisLimitsS4(myAxesS6[2,0],myScale),fontsize=fsz*3)
    myAxesS6[2,2].annotate('i',xy=getAxisLimitsS4(myAxesS6[2,1],myScale),fontsize=fsz*3)
        
    #Remove the ticks
    for cx in range(0,3):
        for cy in range(0,3):
            myAxesS6[cx,cy].set_yticklabels([])
            myAxesS6[cx,cy].set_xticklabels([])
    
    scaleBar = patches.Rectangle((664,814),160,30,linewidth=0,edgecolor='k',facecolor='k')
    myAxesS6[2,2].add_patch(scaleBar)
    myAxesS6[2,2].annotate('6 $\mu$m',xy=[664,900],fontsize=fsz)
    
    myFigS6.tight_layout(pad=0,h_pad=0,w_pad=0)
    plt.show()

In [ ]:
# Generate some 1D data for demonstrating the Hanning bump TIE

# Set up default values 

k0 = (2*np.pi)/0.530 
myC = 0.01
mySNR = 8
w0 = np.pi/31
hanScale = 1e-1
myDim = 256

# set up 1D Fourier Domain TIE solver
kx = np.arange(-np.pi,np.pi,np.pi/256)
phi = k0 / (4*np.pi*(kx**2))
kx = np.arange(-np.pi,np.pi,np.pi/256)
w = np.arange(-np.pi,np.pi,np.pi/256)
han = (hanScale/2)*(1-np.cos((np.pi*w/w0)-np.pi))

for c in range(len(kx)):
    if np.abs(kx[c]) >= w0:
        han[c] = 0

# Generate some pseudo-noise        
np.random.seed(32)
myNoise = np.random.random(len(kx))

# Generate a Gaussian spike signal
myA = np.std(myNoise) * mySNR
mySignal = np.zeros(len(kx))
x = np.arange(-100,100,200/512)
mySignal = myA*np.exp(-(x)**2/(2*myC**2))

mySignalNoise = mySignal + myNoise
mySN = np.fft.fft(mySignalNoise)

phi1 = k0 / ((4*np.pi**2)*(kx**2+1e-12))
phi2 = k0 / ((4*np.pi**2)*(kx**2+han))
phi3 = k0 / ((4*np.pi**2)*(kx**2+hanScale))

# Multiply signal+noise with FD TIE solver

# w/ Hanning bump
myPhase1 = np.fft.ifft(mySN*(phi2))
myNoise1 = np.fft.ifft(np.fft.fft(myNoise)*np.fft.fftshift(phi2))
# w/ Global offset
myPhase2 = np.fft.ifft(mySN*(phi3))
myNoise2 = np.fft.ifft(np.fft.fft(myNoise)*np.fft.fftshift(phi3))


In [ ]:
# Generate Figure S3: Offset Methods for Stabilizing  the 1D TIE in the Fourier Domain

myFig, myAxes = plt.subplots(2,2, figsize=(24,24),dpi=myDPI)


# Default values for 1D TIE
w0 = 1/(3*np.pi)
hanScale = 1.5e-2

#font size
fsz = 21
# Line width
myLW = 4.25

if (1):
    plt.subplots_adjust(left=0.2, bottom=0.2)

    axfreq = plt.axes([0.2, 0.05, 0.65, 0.03])#, facecolor=axcolor)
    axamp = plt.axes([0.2, 0.1, 0.65, 0.03])

    sfreq1 = Slider(axfreq, 'Width (radians)', 5e-3, 0.25, valinit = w0)
    samp1 = Slider(axamp, 'Amplitude', 1e-5, 0.033, valinit = hanScale)

    sfreq1.on_changed(updateFigS3)
    samp1.on_changed(updateFigS3)

# Display Figure S3
updateFigS3(1)
plt.show()
#myFig.savefig('figS31Dx.png')


In [ ]:
# Generate Fig S4: Exploring Offset Methods for Stabilizing TIE Solutions for a 2D Rod Phantom
# Create a microtubule phantom for playing around with Hanning bump characteristics

myLambda = 0.530
px = 0.0375000
dz = 0.600
k0 = 2*np.pi/myLambda
nf = 1e1


hannScale = 1.3e-1
w0 = 4.2e-1 #np.sqrt(1/2) 

myPhantDim = 384
myPhantPhi = np.zeros([myPhantDim,myPhantDim])
myPhantPhi[(int(myPhantDim/2)-52):(int(myPhantDim/2)+52),int(myPhantDim/2)] = 1.8
myPhantImage = np.ones([myPhantDim,myPhantDim])
myPhantNoise1 = np.random.random(np.shape(myPhantImage))
myPhantNoise2 = np.random.random(np.shape(myPhantImage))
myPhantImage = myPhantImage * np.mean(myPhantNoise1)


myNewImgA = TIEProp(myPhantImage,1e3*dz/2,k0,myPhantPhi) 
myNewImgA = myNewImgA - np.min(myNewImgA)
myNewImgB = TIEProp(myPhantImage,-1e3*dz/2,k0,myPhantPhi) 
myNewImgB = myNewImgB - np.min(myNewImgB)


myNewImgAN = myNewImgA +nf*myPhantNoise1
myNewImgBN = myNewImgB + nf*myPhantNoise2

myPhantDI = k0*(myNewImgAN - myNewImgBN) / (dz)
mySigDI = k0*(myNewImgA - myNewImgB) / (dz)
myNoiseDI = k0*(myPhantNoise2 - myPhantNoise1) / (dz)
dI = myPhantDI


myFigS4, myAxesS4 = plt.subplots(2,2, figsize=(24,24), dpi=myDPI)
 
# Set up the sliders for adjusting TIE parameters
axfreq = plt.axes([0.25, 0.05, 0.55, 0.03])#, facecolor=axcolor)
axamp = plt.axes([0.25, 0.1, 0.55, 0.03])
sfreq2 = Slider(axfreq, 'Width (radians)', 0.001, 1.25, valinit=w0)
samp2 = Slider(axamp, 'Amplitude',0.00001, .99, valinit = hannScale)
#room for sliders
plt.subplots_adjust(left=0.1, bottom=-8.50)

sfreq2.on_changed(updateTIEFigS4)
samp2.on_changed(updateTIEFigS4)
# Display Figure S2
updateTIEFigS4(1)
#plt.show()

In [ ]:
# Generate Fig. S5 Impact of Parameter Choices on TIE Phase Micrograph of MTs

myFigS5, myAxesS5 = plt.subplots(2,2, figsize=(24,24), dpi=myDPI)

# Set up initial parameters
myLambda = 0.530
px = 0.0375000
hannScale = 5e-2
w0 = 1.12 #np.sqrt(1/2) 

# Load sample images
if(0):
    Ia = 1.0*(misc.imread('Ia1.tif'))[0:964,0:964]
    Ib = 1.0*(misc.imread('Ib1.tif'))[0:964,0:964]
    dz = 0.045*8
elif(1):
    Ia = 1.0*(misc.imread('Ia2.tif'))[0:964,0:964]
    Ib = 1.0*(misc.imread('Ib2.tif'))[0:964,0:964]
    dz = 0.045*8
elif(1):
    Ia = 1.0*(misc.imread('Ia3.tif'))[0:964,0:964]
    Ib = 1.0*(misc.imread('Ib3.tif'))[0:964,0:964]
    dz = 0.045*8
elif(1):
    Ia = 1.0*(misc.imread('Ia4.tif'))[0:964,0:964]
    Ib = 1.0*(misc.imread('Ib4.tif'))[0:964,0:964]
    dz = 0.045*8

# Calculate first derivative of intensity with respect to defocus
dI = k0 * (Ia-Ib)/(dz)

# Set up the sliders for adjusting TIE parameters
plt.subplots_adjust(left=0.1, bottom=0.2)  
axfreq = plt.axes([0.3, 0.1, 0.55, 0.03])#, facecolor=axcolor)
axamp = plt.axes([0.3, 0.15, 0.55, 0.03])
sfreq2 = Slider(axfreq, 'Width (radians)', np.pi/50, np.pi, valinit=w0)
samp2 = Slider(axamp, 'Amplitude',0.00001, .33, valinit = hannScale)
# Sliders call update function when changed
sfreq2.on_changed(updateTIEFigS5)
samp2.on_changed(updateTIEFigS5)
#
# Call initial update to populate figure S5
updateTIEFigS5(1)



In [ ]:
#Generate Fig. S6: Comparison of VE-BF, TIEM, and Additional Computational Imaging Modes for Visualizing MTs",fontsize=fsz+10)
    
myFigS6, myAxesS6 = plt.subplots(3,3, figsize=(24,24), dpi=myDPI)#*1.250)
# Set up initial parameters
if (1):
    dz = 0.256
    myLambda = 0.530
    k0 = (2*np.pi)/myLambda
    px = 0.0375000
    hannScale = 5e-2
    w0 = 0.45
elif (1):
    dz = 256
    myLambda = 530
    k0 = (2*np.pi)/myLambda
    px = 37.5000
    hannScale = px*2e0
    w0 = px*7.2
elif (1): 
    dz = 0.256e-6
    myLambda = 0.530e-6
    k0 = (2*np.pi)/(myLambda*10**-6)
    px = 0.0375000e-6
    hannScale = 0.9
    w0 = px*12

fsz = 27
myStretch = 0.9

# Load sample images. Choose from one of the following three sample z-stacks, or use your own. 
#
myChoice = 0
if(myChoice == 0):
    Ia = 1.0*(misc.imread('IaCross.tif')) 
    Ib = 1.0*(misc.imread('IbCross.tif')) 
    I0 = 1.0*(misc.imread('I0Cross.tif'))
    myBG = 1.0*(misc.imread('myBGCross.tif'))
elif(myChoice == 1):
    Ia = 1.0*(misc.imread('IaFOV5.tif'))#[200:700,460:960] 
    Ib = 1.0*(misc.imread('IbFOV5.tif'))#[200:700,460:960] 
    I0 = 1.0*(misc.imread('I0FOV5.tif'))#[200:700,460:960]
    myBG = 1.0*(misc.imread('myBGFOV5.tif'))#[200:700,460:960]
    hannScale = 9e-1
    w0 = 0.75
elif(1):
    Ia = 1.0*(misc.imread('Ia8.tif'))[:,328:1292] 
    Ib = 1.0*(misc.imread('Ib88.tif'))[:,328:1292]  
    I0 = 1.0*(misc.imread('I08.tif'))[:,328:1292] 
    myBG = 1.0*(misc.imread('myBG8.tif'))[:,328:1292] 
    hannScale = 9e-2
    w0 = 0.75
    
# Calculate first derivative of intensity with respect to defocus
# dI = np.mean((Ia+Ib)/2)*
dI = (k0  * (Ia-Ib)/(dz) )/(I0+1e-10)


updateTIEFigS6(hannScale,w0)

<strong>Supplementary References</strong>
1. Bormuth, V., Howard, J. & Schäffer, E. LED illumination for video-enhanced DIC imaging of single microtubules. J. Microsc. 226, 1–5 (2007).